# Determine the Fraction of Secondary Structure Elements (SSE)

With protein-quest it is possible to filter on the fraction of Secondary Structure Elements (SSE). It is recommended to not use this filter to strictly since this might result in the loss of too many targets. As an example, let's say you want to filter on a structure having a majority of alpha-helices. This could be done using the `--ratio-min-helix-residues` parameter in the [`protein-quest filter secondary-structure`](cli.html#filter) command. Setting the parameter to 0.5 will keep targets that have more than 50% alpha helices in their structure and filter the ones with less than 50% out.

If you do not know whether there is a major fraction of a certain SSE in your EM density map, than you can make an estimation by using a tool that can identify SSEs in EM density maps of medium to low resolution. A tool that we recommend is the [Emap2sec+](https://kiharalab.org/emsuites/emap2secplus.php) tool from the Kihara lab, which is available in their [webservice](https://em.kiharalab.org/algorithm/emap2sec+).

For this example we will use [EMD-30324](https://www.ebi.ac.uk/emdb/EMD-30324?tab=overview), an EM density map of the human GABA(B) receptor. The build protein model [7CA5](https://www.rcsb.org/structure/7CA5) consist of mainly alpha helical elements. The density has a resolution of 7.6Å. The recommend contour level of the density map is 0.0135 and can be found on the [validation](https://www.ebi.ac.uk/emdb/EMD-30324?tab=validation) tab under Map parameters


Running the job on the Emap2sec+ webservice only requires the EM density map and the contour level. Note that the map downloaded from the EMDB is gzipped and the webservice only accepts `*.map` or `*.mrc`. You have to therefore gunzip the downloaded map before you can upload it on their service.

![Screenshot of input on the Emap2sec+ webservice https://em.kiharalab.org/algorithm/emap2sec+](./SSE_job.jpg)

Once the job is done, the viewer will show which SSEs have been detected, with alpha helices  in red, beta strands in yellow, coil regions in green, and DNA/RNA in purple.
In this job it is clear to see in the viewer that the prediction of SSEs in the density consists mainly of alpha helices

![Screenshot of output on the Emap2sec+ webservice https://em.kiharalab.org/algorithm/emap2sec+](./SSE_output.jpg)

It might however not always be very clear if there is a larger percentage of a certain SSE in an EM density map. To calculate the specific percentages, the script below can be run. The [30324_SSE.pdb](./30324_SSE.pdb) file was extracted as inputFinal_pred.pdb from the downloadable outputs of a Emap2sec+ webservice job.

In [1]:
from pathlib import Path

voxel_counts = {}
total_voxels = 0
SSE_type = {"A": "Other", "B": "Beta sheet", "C": "Alpha helix", "D": "DNA/RNA"}
with Path("30324_SSE.pdb").open() as SSE_file:
    for line in SSE_file:
        if line.startswith("ATOM"):
            chain_id = line[21].strip()
            voxel_counts[chain_id] = voxel_counts.get(chain_id, 0) + 1
            total_voxels += 1

for chain_id, count in voxel_counts.items():
    percentage = (count / total_voxels) * 100
    print(f"{SSE_type[chain_id]}: {count} voxels, {percentage:.1f}% of total")

Alpha helix: 6065 voxels, 56.6% of total
Other: 2110 voxels, 19.7% of total
Beta sheet: 2542 voxels, 23.7% of total
DNA/RNA: 7 voxels, 0.1% of total
